# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [ ]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [9]:
# TODO: Import the necessary libs
# For example: 
import os

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool
import chromadb
import openai
from pydantic import BaseModel
from tavily import TavilyClient

In [5]:
# TODO: Load environment variables
# load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
CHROMA_OPENAI_API_KEY = os.getenv("CHROMA_OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
openai.api_base = os.getenv("OPENAI_BASE_URL") 

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [50]:
# Complete: Create retrieve_game tool
# It should use chroma client and collection you created
from chromadb.utils import embedding_functions

chroma_client = chromadb.PersistentClient(path="chromadb")

# Use the same embedding function as in Udaplay_01
embedding_fn = embedding_functions.OpenAIEmbeddingFunction()
collection = chroma_client.get_collection("udaplay-2", embedding_function=embedding_fn)

# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry. 
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game

@tool
def retrieve_game(query: str) -> list:
    """Semantic search: Finds most results in the vector DB
    args:
    - query: a question about game industry.
    
    You'll receive results as list. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game

    """
    results = collection.query(
        query_texts=[query],
        n_results=5
    )
    return results

#### Evaluate Retrieval Tool

In [51]:
# Complete: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result

class EvaluationModel(BaseModel):
    useful: bool
    description: str

@tool
def evaluate_retrieval(question: str, retrieved_docs: list) -> dict:
    """
    Based on the user's question and on the list of retrieved documents,
    it will analyze the usability of the documents to respond to that question.
    args:
    - question: original question from user
    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
    The result includes:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result

    """

    evaluation_prompt = f"""Validate if the following list of retrieved documents are useful to answer the user's query, including a detailed explanation.

    User's query: {question}
    
    Retrieved_docs: {retrieved_docs}

    Assess whether these documents contain sufficient information to answer the question. Return your answer with a boolean 'useful' field and a detailed 'description' explaining your reasoning."""

    # Create LLM instance and invoke with structured output
    llm = LLM(model="gpt-4o-mini", temperature=0)
    result = llm.invoke(evaluation_prompt, EvaluationModel)
    return result.dict()

#### Game Web Search Tool

In [46]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry. 

@tool
def game_web_search(question: str) -> list:
  """
  Web search: find results from the web about a game
  args:
  - question: a question about game industry.

  """
  tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

  search_results = tavily_client.search(
    query=question,
    max_results=5,
  )

  return search_results.get("results", [])


### Agent

In [52]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed
from lib.state_machine import StateMachine, Step, EntryPoint, Termination, Resource
from typing import TypedDict

instructions = """
You are a research agent for the video game industry. Users will ask you questions about video games.
When you receive a question, first run a semantic search to try to find relevant info, and then
evaluate the results using an evaluation tool to check if the info retrieved is sufficient
to answer the original question. If the retrieval results are evaluated as not good enough, to a web search
with the question to up-to-date info to construct your answer to the question 

"""

agent = Agent(
  model_name="gpt-4o-mini",
  instructions=instructions,
  tools=[retrieve_game, evaluate_retrieval, game_web_search],
  temperature=0
)



In [53]:
# Complete: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?

agent.reset_session()
run1 = agent.invoke("When Pokémon Gold and Silver was released?")
run2 = agent.invoke("Which one was the first 3D platformer Mario game?")
run3 = agent.invoke("Was Mortal Kombat X realeased for Playstation 5?")

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor


In [33]:
def print_run_summary(run, run_name="Run"):
    """Helper function to print the results of Agent runs"""
    print(f"=== {run_name} Summary ===")
    
    # Get all snapshots to see the progression
    snapshots = run.snapshots
    
    print(f"Total steps: {len(snapshots)}")
    print()
    
    # Track the steps
    for i, snapshot in enumerate(snapshots):
        step_name = snapshot.step_id
        state = snapshot.state_data
        
        print(f"Step {i + 1}: {step_name}")
        
        # Show any tool calls made in this step
        if state.get("current_tool_calls"):
            for call in state["current_tool_calls"]:
                print(f"  - Tool called: {call.function.name}")
        
        # Show AI messages generated in this step
        messages = state.get("messages", [])
        if messages:
            last_msg = messages[-1]
            if isinstance(last_msg, AIMessage):
                content_preview = last_msg.content[:100] + "..." if (last_msg.content and len(last_msg.content) > 100) else last_msg
                print(f"  - AI response: {content_preview}")
            elif isinstance(last_msg, ToolMessage):
                print(f"  - Tool result: {last_msg.content if last_msg.content else last_msg.name}")
        print()
    
    # Final response
    final_state = run.get_final_state()
    if final_state and "messages" in final_state:
        final_message = final_state["messages"][-1]
        if isinstance(final_message, AIMessage):
            print("=== Final Response ===")
            print(final_message.content)
        
        # Show token usage if available
        total_tokens = final_state.get("total_tokens", 0)
        if total_tokens > 0:
            print(f"\nTotal tokens used: {total_tokens}")
    
    print("=" * 50)
    print()

In [54]:
print_run_summary(run1, "Pokémon Gold/Silver Release")
print_run_summary(run2, "First 3D Mario Game")
print_run_summary(run3, "Mortal Kombat X on PS5")

=== Pokémon Gold/Silver Release Summary ===
Total steps: 7

Step 1: __entry__

Step 2: message_prep

Step 3: llm_processor
  - Tool called: retrieve_game
  - AI response: role='assistant' content=None tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_sxidjESvyudJCUNCcka465t9', function=Function(arguments='{"query":"Pokémon Gold and Silver release date"}', name='retrieve_game'), type='function')] token_usage=None

Step 4: tool_executor
  - Tool result: "{'ids': [['006', '007', '012', '009', '008']], 'embeddings': None, 'documents': [['[Game Boy Color] Pok\u00e9mon Gold and Silver (1999) - Second-generation Pok\u00e9mon games introducing new regions, Pok\u00e9mon, and gameplay mechanics.', '[Game Boy Advance] Pok\u00e9mon Ruby and Sapphire (2002) - Third-generation Pok\u00e9mon games set in the Hoenn region, featuring new Pok\u00e9mon and double battles.', '[Nintendo Switch] Mario Kart 8 Deluxe (2017) - An enhanced version of Mario Kart 8, featuring new characters, tracks, and i

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes